In [44]:
import os
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
from dotenv import load_dotenv

In [45]:
# Load environment variables from .env file
load_dotenv()

True

In [46]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)


In [47]:
with open('mockbin_api.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()

In [48]:
# Initialize agent OpenAPI tool using the read in OpenAPI spec
openapi = OpenApiTool(name="get_catfacts", spec=openapi_spec, description="Facts about cats", auth=auth)

In [ ]:
# Create agent with OpenAPI tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["O_MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=openapi.definitions
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Tell me a cat fact",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_atIZZH4vjfyIb7YxUeSvGlgl
Created thread, ID: thread_CAZF5fVs3tPnkpAAKZumoAEs
Created message, ID: msg_vkh1qRpkYUGgqAI973ReSPaa
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_4usucAgB8yd03zMz8XRTtxuZ', 'object': 'thread.message', 'created_at': 1740681694, 'assistant_id': 'asst_atIZZH4vjfyIb7YxUeSvGlgl', 'thread_id': 'thread_CAZF5fVs3tPnkpAAKZumoAEs', 'run_id': 'run_XerbXGJIcorfiJAcWciFlU3P', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "It seems there was an error when attempting to retrieve the cat fact. Unfortunately, without additional details about the error, it's difficult to pinpoint the exact cause. Possible reasons could include:\n\n1. Network issues or connectivity problems.\n2. The API endpoint might be temporarily down or unavailable.\n3. There could be an issue with the parameters sent in the request.\n4. There might be restrictions or limitations on the API usage.\n